In [1]:
# Khai báo các thư viện cần thiết
import os
import random
import pickle

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


In [2]:
# Lớp TrainingData 
# Lớp này dùng để lưu trữ dữ liệu huấn luyện cho mô hình

class TrainingData:
    def __init__(self):
        """
        Khởi tạo lớp TrainingData với dữ liệu và nhãn.
        
        :param data: Dữ liệu huấn luyện (danh sách các mẫu).
        :param labels: Nhãn tương ứng với dữ liệu (danh sách các nhãn).
        """
        self.data = []
    
    def add(self, sample):
        """
        Thêm một mẫu vào dữ liệu huấn luyện.
        
        :param sample: Mẫu dữ liệu cần thêm.
        """
        self.data.append(sample)

    def sample(self, batch_size):
        """
        Lấy một mẫu ngẫu nhiên từ dữ liệu huấn luyện với kích thước batch_size.
        
        :param batch_size: Kích thước của mẫu cần lấy.
        :return: Một danh sách chứa các mẫu ngẫu nhiên.
        """
        return random.sample(self.data, batch_size) if len(self.data) >= batch_size else self.data
    
    def __len__(self):
        """
        Trả về số lượng mẫu trong dữ liệu huấn luyện.
        
        :return: Số lượng mẫu trong dữ liệu.
        """
        return len(self.data)

In [3]:
# Nạp dữ liệu huấn luyện vào training_data
training_data = TrainingData()
data_dir = "data"  # Thư mục chứa dữ liệu huấn luyện

label = 0  # Biến nhãn khởi tạo bằng 0
label_names = []  # Danh sách chứa tên nhãn
# Với mỗi thư mục con trong thư mục data, lấy dữ liệu cho vào training_data đồng thời gán nhãn là số thứ tự của thư mục
for subdir in os.listdir(data_dir):
    subdir_path = os.path.join(data_dir, subdir)
    if os.path.isdir(subdir_path):
        label_names.append(subdir)
        print(f"Loading data from {subdir_path}...")
        for filename in os.listdir(subdir_path):
            file_path = os.path.join(subdir_path, filename)
            if os.path.isfile(file_path):
                with open(file_path, 'rb') as f:
                    data = pickle.load(f)
                    data['label'] = label  # Gán nhãn cho dữ liệu
                    training_data.add(data)
        label += 1  # Tăng nhãn cho thư mục tiếp 
        
print(f"Loaded {len(training_data)} samples from {len(label_names)} classes.")

Loading data from data\test...
Loaded 1 samples from 1 classes.


In [4]:
# Mạng nơron

class MetaModelNet(nn.Module):
    """
    Mạng nơron cho mô hình MetaModelNet.
    """
    def __init__(self, local_size, num_classes):
        super(MetaModelNet, self).__init__()
    
        # Tầng CNN cho local_obs
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)

        # Tầng FC cho position
        self.fc_position = nn.Linear(2, 32)

        # Tầng FC cuối cùng
        self.fc1 = nn.Linear(256 * local_size * local_size + 32, 512)
        self.fc2 = nn.Linear(512, 256)
        
        # Dropout trước tầng FC cuối cùng
        self.dropout = nn.Dropout(0.5)
        # Tầng đầu ra
        self.fc3 = nn.Linear(256, num_classes)

    def forward(self, local_obs, position):
        """
        Hàm truyền dữ liệu qua mạng nơron.
        
        :param local_obs: Dữ liệu đầu vào (local observations).
        :param position: Vị trí của dữ liệu đầu vào.
        :return: Kết quả đầu ra của mạng nơron.
        """
        # Xử lý local_obs qua các tầng CNN
        x_local = F.relu(self.conv1(local_obs))
        x_local = F.relu(self.conv2(x_local))
        x_local = F.relu(self.conv3(x_local))
        x_local = F.relu(self.conv4(x_local))

        # Chuyển đổi kích thước tensor
        x_local = x_local.view(x_local.size(0), -1)  # Chuyển đổi thành vector 1 chiều

        # Xử lý position qua tầng FC
        x_position = F.relu(self.fc_position(position))

        # Kết hợp dữ liệu từ local_obs và position
        x = torch.cat((x_local, x_position), dim=1)

        # Truyền qua các tầng FC
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x        

In [ ]:
# Khởi tạo các siêu tham số

local_size = 11  # Kích thước của local_obs
num_classes = len(label_names)  # Số lớp (số nhãn)

max_episodes = 1000  # Số lượng episode tối đa

batch_size = 32  # Kích thước batch

learning_rate = 0.001  # Tốc độ học
weight_decay = 0.0001  # Tham số weight decay

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Kiểm tra xem có GPU hay không
print(f"Using device: {device}")

# Khởi tạo mô hình, hàm mất mát và bộ tối ưu hóa
model = MetaModelNet(local_size, num_classes).to(device)

optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

Using device: cuda


In [ ]:
# Vòng lặp huấn luyện
for episode in range(max_episodes):
    #1. Lấy một batch dữ liệu ngẫu nhiên từ training_data
    batch = training_data.sample(batch_size)

    #2. Tách dữ liệu thành các phần riêng biệt
    local_obs = torch.stack([torch.tensor(sample['local_obs'], dtype=torch.float32) for sample in batch])
    agent_start_positions = torch.stack([torch.tensor(sample['agent_start_position'], dtype=torch.float32) for sample in batch])
    agent_end_positions = torch.stack([torch.tensor(sample['agent_end_position'], dtype=torch.float32) for sample in batch])
    labels = torch.tensor([sample['label'] for sample in batch], dtype=torch.long)

    #3. Chuyển dữ liệu sang GPU nếu có
    local_obs = local_obs.to(device)
    agent_start_positions = agent_start_positions.to(device)
    agent_end_positions = agent_end_positions.to(device)
    labels = labels.to(device)

    #4. Tính giá trị target dựa vào agent_start_positions và agent_end_positions
    target_positions = agent_end_positions - agent_start_positions  
    target = torch.sum(target_positions, dim=1)  # Tính tổng các vị trí

    #5. Đưa dữ liệu vào mô hình và tính toán dự đoán
    local_obs = local_obs.unsqueeze(1)  # Thêm chiều cho local_obs
    predictions = model(local_obs, agent_start_positions)
    prediction = predictions[torch.arange(batch_size), labels]
    
    #6. Tính toán hàm mất mát
    loss = F.mse_loss(prediction, target)
    print(f"prediction: {prediction}")
    print(f"target: {target}")
    print(f"loss: {loss.item()}")
    
    #7. Tối ưu hóa mô hình
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    


prediction: tensor([0.0487], device='cuda:0', grad_fn=<IndexBackward0>)
target: tensor([11.], device='cuda:0')
loss: 119.93196868896484
prediction: tensor([0.0524], device='cuda:0', grad_fn=<IndexBackward0>)
target: tensor([11.], device='cuda:0')
loss: 119.84992980957031
prediction: tensor([0.0480], device='cuda:0', grad_fn=<IndexBackward0>)
target: tensor([11.], device='cuda:0')
loss: 119.94646453857422
prediction: tensor([0.0234], device='cuda:0', grad_fn=<IndexBackward0>)
target: tensor([11.], device='cuda:0')
loss: 120.48588562011719
prediction: tensor([0.0266], device='cuda:0', grad_fn=<IndexBackward0>)
target: tensor([11.], device='cuda:0')
loss: 120.41500854492188
prediction: tensor([0.0722], device='cuda:0', grad_fn=<IndexBackward0>)
target: tensor([11.], device='cuda:0')
loss: 119.41687774658203
prediction: tensor([0.0725], device='cuda:0', grad_fn=<IndexBackward0>)
target: tensor([11.], device='cuda:0')
loss: 119.410400390625
prediction: tensor([0.0893], device='cuda:0', grad